In [ ]:
## The code in this notebook ist in part taken from this repository.
## Credit to: F.  Radenovic,  A.  Iscen,  G.  Tolias,  Y.  Avrithis,  and  O.  Chum.  Revisitingoxford  and  paris:   Large-scale  image  retrieval  benchmarking.   InCVPR,2018.
!rm -rf revisitop
!git clone https://github.com/filipradenovic/revisitop.git
data_root = 'revisitop/data/'
from revisitop.python.download import download_datasets
download_datasets(data_root)

Cloning into 'revisitop'...
remote: Enumerating objects: 168, done.
remote: Total 168 (delta 0), reused 0 (delta 0), pack-reused 168
Receiving objects: 100% (168/168), 34.17 KiB | 1.42 MiB/s, done.
Resolving deltas: 100% (80/80), done.
>> Dataset roxford5k directory does not exist. Creating: revisitop/data/datasets/roxford5k/jpg
>> Downloading dataset roxford5k archive oxbuild_images.tgz...
>> Extracting dataset roxford5k archive oxbuild_images.tgz...
>> Extracted, deleting dataset roxford5k archive oxbuild_images.tgz...
>> Downloading dataset roxford5k ground truth file...
>> Dataset rparis6k directory does not exist. Creating: revisitop/data/datasets/rparis6k/jpg
>> Downloading dataset rparis6k archive paris_1.tgz...
>> Extracting dataset rparis6k archive paris_1.tgz...
>> Extracted, deleting dataset rparis6k archive paris_1.tgz...
>> Downloading dataset rparis6k archive paris_2.tgz...
>> Extracting dataset rparis6k archive paris_2.tgz...
>> Extracted, deleting dataset rparis6k archi

In [ ]:
!rm -rf functions
!git clone https://github.com/Lennard-Alms/lab_bd.git functions

Cloning into 'functions'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (231/231), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 974 (delta 138), reused 164 (delta 73), pack-reused 743
Receiving objects: 100% (974/974), 692.59 KiB | 6.72 MiB/s, done.
Resolving deltas: 100% (597/597), done.


In [ ]:
import os
import numpy as np
from PIL import Image, ImageFile
from functions.preprocessing.BuildDB import extract_features
from functions.model.CNN_Strats import build_model
from functions.preprocessing.HelperFunctions import get_image, pil_loader
from revisitop.python.evaluate import compute_map
from revisitop.python.dataset import configdataset
import matplotlib.pyplot as plt
import numpy as np
import gc
import cv2
import tensorflow as tf

In [ ]:
data_root = 'revisitop/data/'
test_dataset = 'roxford5k'
cfg = configdataset(test_dataset, os.path.join(data_root, 'datasets'))
gnd = cfg['gnd']
ks = [1, 5, 10]

In [ ]:
max_shape = (1024, 1024, 3)
EXP = [9]
DEP = [3,4,5]
with open('output.txt', 'w') as f:
  f.write('VGG RMAC \n')

In [ ]:
for exp in EXP:
  for dep in DEP: 
    imodel = build_model(max_shape, exp, vgg_output=True, rmac=True, depth=dep)
    im_paths = np.array([cfg['im_fname'](cfg, i) for i in np.arange(cfg['n'])])
    im_features = extract_features(im_paths, imodel, batch_size=20, network_batch_size=4, pil=True, shape=max_shape, fill_option='white', concat=True)
    q_features = []
    for i in np.arange(cfg['nq']):
      qim = np.array(pil_loader(cfg['qim_fname'](cfg, i)).crop(gnd[i]['bbx']))
      qmodel = build_model(qim.shape, exp, vgg_output=True, mac=True)
      q_features.append(extract_features([qim], qmodel, batch_size=100, network_batch_size=4))
    q_features = np.concatenate(q_features)
    dist = np.arccos(np.dot(im_features, q_features.T))
    ranks = np.argsort(dist, axis=0)
    # search for easy
    gnd_t = []
    for i in range(len(gnd)):
        g = {}
        g['ok'] = np.concatenate([gnd[i]['easy']])
        g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['hard']])
        gnd_t.append(g)
    mapE, apsE, mprE, prsE = compute_map(ranks, gnd_t, ks)

    # search for easy & hard
    gnd_t = []
    for i in range(len(gnd)):
        g = {}
        g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
        g['junk'] = np.concatenate([gnd[i]['junk']])
        gnd_t.append(g)
    mapM, apsM, mprM, prsM = compute_map(ranks, gnd_t, ks)

    # search for hard
    gnd_t = []
    for i in range(len(gnd)):
        g = {}
        g['ok'] = np.concatenate([gnd[i]['hard']])
        g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
        gnd_t.append(g)
    mapH, apsH, mprH, prsH = compute_map(ranks, gnd_t, ks)
    with open('output.txt', 'a') as f:
      f.write('=========================================== \n')
      f.write('vgg rmac exp:' + str(exp) + ' dep:' + str(dep) + ' \n')
      f.write('>> {}: mAP E: {}, M: {}, H: {} \n'.format(test_dataset, np.around(mapE*100, decimals=2), np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
      f.write('>> {}: mP@k{} E: {}, M: {}, H: {} \n'.format(test_dataset, np.array(ks), np.around(mprE*100, decimals=2), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))
    print('>> {}: mAP E: {}, M: {}, H: {}'.format(test_dataset, np.around(mapE*100, decimals=2), np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
    print('>> {}: mP@k{} E: {}, M: {}, H: {}'.format(test_dataset, np.array(ks), np.around(mprE*100, decimals=2), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))

1/1 [==============================] - 4s 4s/step
>> roxford5k: mAP E: 53.63, M: 33.21, H: 5.78
>> roxford5k: mP@k[ 1  5 10] E: [77.94 69.26 63.97], M: [75.71 57.71 49.89], H: [24.29 16.86 14.46]
1/1 [==============================] - 0s 194ms/step
>> roxford5k: mAP E: 41.9, M: 26.53, H: 4.45
>> roxford5k: mP@k[ 1  5 10] E: [66.18 57.38 53.11], M: [65.71 50.86 43.71], H: [14.29 10.    9.14]
1/1 [==============================] - 0s 192ms/step
>> roxford5k: mAP E: 34.77, M: 22.56, H: 3.51
>> roxford5k: mP@k[ 1  5 10] E: [54.41 45.88 42.65], M: [55.71 41.71 36.57], H: [8.57 7.43 7.  ]


In [ ]:
with open('output.txt', 'r') as f:
  print(f.read())

VGG RMAC 
vgg rmac exp:9 dep:3 
>> roxford5k: mAP E: 53.63, M: 33.21, H: 5.78 
>> roxford5k: mP@k[ 1  5 10] E: [77.94 69.26 63.97], M: [75.71 57.71 49.89], H: [24.29 16.86 14.46] 
vgg rmac exp:9 dep:4 
>> roxford5k: mAP E: 41.9, M: 26.53, H: 4.45 
>> roxford5k: mP@k[ 1  5 10] E: [66.18 57.38 53.11], M: [65.71 50.86 43.71], H: [14.29 10.    9.14] 
vgg rmac exp:9 dep:5 
>> roxford5k: mAP E: 34.77, M: 22.56, H: 3.51 
>> roxford5k: mP@k[ 1  5 10] E: [54.41 45.88 42.65], M: [55.71 41.71 36.57], H: [8.57 7.43 7.  ] 



>> roxford5k: mAP E: 68.25, M: 48.68, H: 20.54
>> roxford5k: mP@k[ 1  5 10] E: [92.65 84.31 75.78], M: [90.   78.57 69.29], H: [51.43 39.14 32.57]


In [ ]:
old mac 3 vgg
>> roxford5k: mAP E: 58.3, M: 40.65, H: 15.3
>> roxford5k: mP@k[ 1  5 10] E: [80.88 69.56 65.59], M: [80.   64.1  57.95], H: [40.   29.57 24.86]

rmac 3 vgg 1
>> roxford5k: mAP E: 54.27, M: 37.11, H: 13.09
>> roxford5k: mP@k[ 1  5 10] E: [79.41 68.9  64.31], M: [77.14 62.1  55.24], H: [34.29 25.86 22.71]

rmac 3 vgg 2
>> roxford5k: mAP E: 56.51, M: 35.63, H: 7.49
>> roxford5k: mP@k[ 1  5 10] E: [82.35 73.46 66.1 ], M: [80.   62.   52.98], H: [34.29 22.   17.52]

rmac 3 vgg 3
>> roxford5k: mAP E: 56.36, M: 34.88, H: 6.36
>> roxford5k: mP@k[ 1  5 10] E: [82.35 73.9  67.28], M: [81.43 61.14 52.71], H: [31.43 18.86 14.29]

rmac 10 vgg 2
>> roxford5k: mAP E: 53.04, M: 34.8, H: 8.32
>> roxford5k: mP@k[ 1  5 10] E: [76.47 65.88 61.34], M: [75.71 59.52 53.88], H: [30.   24.21 19.7 ]

rmac 10 vgg 3
>> roxford5k: mAP E: 55.64, M: 34.84, H: 6.58
>> roxford5k: mP@k[ 1  5 10] E: [82.35 73.68 65.59], M: [80.   62.86 52.29], H: [31.43 19.14 15.14]

rmac 15 vgg 3
>> roxford5k: mAP E: 53.85, M: 33.46, H: 6.48
>> roxford5k: mP@k[ 1  5 10] E: [80.88 73.68 65.74], M: [78.57 62.86 52.57], H: [31.43 19.71 15.29]

rmac 15 vgg 4
>> roxford5k: mAP E: 47.2, M: 29.42, H: 4.95
>> roxford5k: mP@k[ 1  5 10] E: [73.53 65.81 60.53], M: [71.43 56.57 49.29], H: [18.57 12.29 10.29]

rmac 10 atten 3 -> q vgg mac
>> roxford5k: mAP E: 57.23, M: 38.03, H: 11.01
>> roxford5k: mP@k[ 1  5 10] E: [82.35 75.07 66.94], M: [80. 66. 56.], H: [31.43 24.57 20.43]

rmac 10 atten 4 -> q vgg mac
>> roxford5k: mAP E: 56.05, M: 37.1, H: 10.47
>> roxford5k: mP@k[ 1  5 10] E: [80.88 72.99 67.18], M: [78.57 64.86 56.57], H: [31.43 22.57 19.14]

rmac 5 atten 3 -> q vgg mac
>> roxford5k: mAP E: 57.95, M: 38.16, H: 10.61
>> roxford5k: mP@k[ 1  5 10] E: [83.82 73.97 68.53], M: [81.43 65.43 57.29], H: [31.43 24.29 19.57]

rmac 5 vgg atten 3 -> q vgg mac
>> roxford5k: mAP E: 57.39, M: 37.71, H: 10.01
>> roxford5k: mP@k[ 1  5 10] E: [80.88 72.06 68.03], M: [78.57 62.95 56.52], H: [30.   25.86 20.29]

rmac 5 atten 2 -> q vgg mac
>> roxford5k: mAP E: 57.08, M: 38.42, H: 11.47
>> roxford5k: mP@k[ 1  5 10] E: [85.29 72.4  66.08], M: [82.86 64.57 57.14], H: [31.43 26.07 20.79]

rmac 3 atten 3 -> q vgg mac
>> roxford5k: mAP E: 55.74, M: 36.44, H: 8.95
>> roxford5k: mP@k[ 1  5 10] E: [86.76 72.28 66.52], M: [84.29 62.67 56.1 ], H: [27.14 20.71 18.  ]

regio 3 vgg 3 -> q vgg mac (158)
>> roxford5k: mAP E: 68.25, M: 48.68, H: 20.54
>> roxford5k: mP@k[ 1  5 10] E: [92.65 84.31 75.78], M: [90.   78.57 69.29], H: [51.43 39.14 32.57]

In [ ]:
im_features_all = []
for batch in range(len(im_features)):
  for im in range(len(im_features[batch][0])):
    all_im = []
    for size in range(len(im_features[0])):
      for y in range(len(im_features[batch][size][im])):
        for x in range(len(im_features[batch][size][im][y])):
          all_im.append(im_features[batch][size][im][y][x])
    im_features_all.append(all_im)
im_features_all = np.array(im_features_all)
final_dists = []
for im_feat in im_features_all:
  dist = np.arccos(np.dot(im_feat, q_features.T))
  sdist = np.sort(dist, axis=0)
  final_dists.append((sdist[0]))
final_dists = np.array(final_dists)
ranks = np.argsort(final_dists, axis=0)

In [ ]:
qmodel = build_model(qavg_shape, 3, vgg_output=False, attention=True, rmac=True)
qims = [np.array(pil_loader(cfg['qim_fname'](cfg, i)).crop(gnd[i]['bbx'])) for i in np.arange(cfg['nq'])]
q_features = extract_features(qims, qmodel, batch_size=100, network_batch_size=4, pil=True, shape=qavg_shape, fill_option='resize')
print(q_features.shape, im_features.shape)

In [ ]:
mx,my,mz = 0,0,0
ax,ay,az = 0,0,0
n = cfg['n']
qn = cfg['nq']

for i in np.arange(n):
  ix,iy,iz = get_image(cfg['im_fname'](cfg, i), pil=True).shape
  mx,my,mz = max(mx,ix), max(my,iy), max(mz,iz)
  ax,ay,az = ax+ix, ay+iy, az+iz
iax,iay,iaz = ax//n, ay//n, az//n
ax,ay,az = 0,0,0

for i in np.arange(qn):
  ix,iy,iz = np.array(pil_loader(cfg['qim_fname'](cfg, i)).crop(gnd[i]['bbx'])).shape
  mx,my,mz = max(mx,ix), max(my,iy), max(mz,iz)
  ax,ay,az = ax+ix, ay+iy, az+iz
qax,qay,qaz = ax//qn, ay//qn, az//qn

max_shape = (mx,my,mz)
iavg_shape = (iax,iay,iaz)
qavg_shape = (qax,qay,qaz)
print(max_shape,iavg_shape,qavg_shape)

(1024, 1024, 3) (850, 921, 3) (554, 478, 3)
